# Bing & Google Image Downloader


In [1]:
import cv2
import os

import requests
from requests import exceptions
from tqdm import tqdm_notebook

# output dir
# dir_output = 'data/желчные'
# os.makedirs(dir_output, exist_ok=True)
# if os.path.isdir(dir_output): print('Working directory:', os.path.abspath(dir_output))


# Google: JavaScript code 

```
urls = Array.from(document.querySelectorAll('.rg_di .rg_meta')).map(el=>JSON.parse(el.textContent).ou);
window.open('data:text/csv;charset=utf-8,' + escape(urls.join('\n')));
```

## Bing: Initialize

In [6]:
# set your Microsoft Cognitive Services API key along with (1) the
# maximum number of results for a given search and (2) the group size
# for results (maximum of 50 per request)
# Ключ 1: 8369e47c2ab54b9a9c4838c9bc9d29ff
# Ключ 2: fa444f77248d4be89ef15449f36970f0 
API_KEY = "8369e47c2ab54b9a9c4838c9bc9d29ff"
MAX_RESULTS = 1000
GROUP_SIZE = 50
 
# set the endpoint API URL
URL = "https://api.cognitive.microsoft.com/bing/v7.0/images/search"

## Bing: Get total number of results

In [17]:
# search term 
term = 'перечный гриб'

# store the search term in a convenience variable then set the
# headers and search parameters
headers = {"Ocp-Apim-Subscription-Key" : API_KEY}
params = {"q": term, "offset": 0, "count": GROUP_SIZE}
 
# make the search
print("Searching Bing API for '{}'".format(term))
search = requests.get(URL, headers=headers, params=params)
search.raise_for_status()
 
# grab the results from the search, including the total number of
# estimated results returned by the Bing API
results = search.json()
estNumResults = min(results["totalEstimatedMatches"], MAX_RESULTS)
print("Total results:", estNumResults)
 

Searching Bing API for 'перечный гриб'
Total results: 948


## Bing: Write URLs to file

In [18]:
f = open('data/bing_перечный.txt', "a+")

for offset in range(0, estNumResults, GROUP_SIZE):
    # update the search parameters using the current offset, then
    # make the request to fetch the results
    print("Request for group {}-{} of {}...".format(offset, offset + GROUP_SIZE, estNumResults))
    params["offset"] = offset
    search = requests.get(URL, headers=headers, params=params)
    search.raise_for_status()
    results = search.json()
    
    # loop over the results
    for v in results["value"]:
        f.write(v["contentUrl"] + '\n')

f.close()
print('Done')

Request for group 0-50 of 948...
Request for group 50-100 of 948...
Request for group 100-150 of 948...
Request for group 150-200 of 948...
Request for group 200-250 of 948...
Request for group 250-300 of 948...
Request for group 300-350 of 948...
Request for group 350-400 of 948...
Request for group 400-450 of 948...
Request for group 450-500 of 948...
Request for group 500-550 of 948...
Request for group 550-600 of 948...
Request for group 600-650 of 948...
Request for group 650-700 of 948...
Request for group 700-750 of 948...
Request for group 750-800 of 948...
Request for group 800-850 of 948...
Request for group 850-900 of 948...
Request for group 900-950 of 948...
Done


## Function to download images

In [14]:
# when attempting to download images from the web both the Python
# programming language and the requests library have a number of
# exceptions that can be thrown so let's build a list of them now
# so we can filter on them
EXCEPTIONS = set([IOError, FileNotFoundError,
    exceptions.RequestException, exceptions.HTTPError,
    exceptions.ConnectionError, exceptions.Timeout])


def download_images_from_urls(filepath_urls, dirpath_dest):
    os.makedirs(dirpath_dest, exist_ok=True)
    print('Destination dir:', os.path.abspath(dirpath_dest))
    
    count_total = 0
    count_deleted = 0
    
    with open(filepath_urls) as file_urls:
        lines = file_urls.readlines()
        
        for line in tqdm_notebook(lines):
            try:
                url = line.rstrip('\n')
                r = requests.get(url, timeout=30, verify=False)
                ext = url[url.rfind("."):]
                if '?' in ext: ext = ext[:ext.find('?')]
                if '/' in ext: ext = ext[:ext.find('/')]
                if '&' in ext: ext = ext[:ext.find('&')]
                ext = ext.lower()
                if ext not in ['.gif', '.jpg', '.jpeg', '.png']: 
                    ext = '.jpg'
                
                p = os.path.sep.join([dirpath_dest, "{}{}".format(str(count_total).zfill(8), ext)])
                # write the image to disk
                f = open(p, "wb")
                f.write(r.content)
                f.close()

                # if os.path.isfile(p): print('File saved:', p)
                # try to load the image from disk
                # image = cv2.imread(p)

            except Exception as e:
                # check exception in our list of exceptions
                print(e)
                if type(e) in EXCEPTIONS: print("Skipping:", url)
                count_deleted += 1

            # update the counter
            count_total += 1
        
        print('Download completed. Total:', count_total, ', deleted:', count_deleted)
        

# Download from file containing URLs  

In [15]:
# from fastai.vision import *
# download_images('data/bing_лисички.txt', dir_dest, max_pics=1000)

import warnings; 
warnings.simplefilter('ignore')

download_images_from_urls('data/bing_опята.txt', 'data/опята_bing')
download_images_from_urls('data/bing_рыжики.txt', 'data/рыжики_bing')
download_images_from_urls('data/bing_маслята.txt', 'data/маслята_bing')
download_images_from_urls('data/bing_желчные.txt', 'data/желчные_bing')

Destination dir: C:\DATA\Projects\mush-room\data\опята_bing


('Connection aborted.', ConnectionResetError(10054, 'Удаленный хост принудительно разорвал существующее подключение', None, 10054, None))
Skipping: http://otvet-plus.ru/iz/opjat-2.jpg
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))
Skipping: https://primamedia.ru/f/big/1094/1093754.jpg
HTTPConnectionPool(host='library.kharkov.ua', port=80): Read timed out.
Skipping: http://library.kharkov.ua/expobook/LibKh00000000233.jpg
('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))
Skipping: https://primamedia.ru/f/big/1094/1093754.jpg
HTTPConnectionPool(host='www.family-port.ru', port=80): Max retries exceeded with url: /RaskraskiSkazkiBabaJaga4.jpg (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000000006B1D390>: Failed to establish a new connection: [WinError 10060] Попытка установить соединение была безуспешной, т.к. от другого компьютера за требуемое время не получен нужный


Download completed. Total: 808 , deleted: 0
Destination dir: C:\DATA\Projects\mush-room\data\маслята_bing


HTTPSConnectionPool(host='pngimg.com', port=443): Max retries exceeded with url: /uploads/mushroom/mushroom_PNG3209.png (Caused by SSLError(SSLError(1, '[SSL: TLSV1_ALERT_PROTOCOL_VERSION] tlsv1 alert protocol version (_ssl.c:833)'),))

Download completed. Total: 998 , deleted: 1
Destination dir: C:\DATA\Projects\mush-room\data\желчные_bing


HTTPConnectionPool(host='spasisebya.ucoz.ru', port=80): Max retries exceeded with url: /Inter/svin.jpg (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000000006B1D710>: Failed to establish a new connection: [WinError 10060] Попытка установить соединение была безуспешной, т.к. от другого компьютера за требуемое время не получен нужный отклик, или было разорвано уже установленное соединение из-за неверного отклика уже подключенного компьютера',))
Skipping: http://spasisebya.ucoz.ru/Inter/svin.jpg
HTTPConnectionPool(host='spasisebya.ucoz.ru', port=80): Max retries exceeded with url: /Inter/svin.jpg (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000000055FE2B0>: Failed to establish a new connection: [WinError 10060] Попытка установить соединение была безуспешной, т.к. от другого компьютера за требуемое время не получен нужный отклик, или было разорвано уже установленное соединение из-за неверного отклика уже подключенного комп

# Rename files

In [2]:
def rename_files_suffix(dirpath, suffix):
    for f in tqdm_notebook(os.listdir(dirpath)):
        filepath = os.path.join(dirpath, f)
        if os.path.isfile(filepath):
            fsplit = os.path.splitext(f)
            os.rename(filepath, os.path.join(dirpath, fsplit[0] + suffix + fsplit[1]))
            
def rename_files_counter(dirpath, prefix):
    contents = os.listdir(dirpath)
    for i, f in tqdm_notebook(enumerate(contents), total=len(contents)):
        filepath = os.path.join(dirpath, f)
        if os.path.isfile(filepath):
            fsplit = os.path.splitext(f)
            os.rename(filepath, os.path.join(dirpath, prefix + str(i).zfill(4) + fsplit[1]))


In [4]:
rename_files_suffix('data/рыжики_google', '_')

In [5]:
rename_files_counter('data/опята', 'опята_')